In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from fastai import *
from fastai.vision import *
from fastai.vision.image import *

In [ ]:
import cv2

# adapted from https://www.kaggle.com/iafoss/pretrained-resnet34-with-rgby-0-460-public-lb
def open_4_channel(fname):
    fname = str(fname)
    suffix = '.png'
    # strip extension before adding color
    if fname.endswith('.png') or fname.endswith('.tif'):
        suffix = fname[-4:]
        fname = fname[:-4]

    colors = ['red','green','blue','yellow']
    flags = cv2.IMREAD_GRAYSCALE
    img = [cv2.imread(fname+'_'+color+suffix, flags).astype(np.float32)/255
           for color in colors]
    
    x = np.stack(img, axis=-1)
    return Image(pil2tensor(x, np.float32).float())

In [ ]:
import torchvision


RESNET_ENCODERS = {
    34: torchvision.models.resnet34,
    50: torchvision.models.resnet50,
    101: torchvision.models.resnet101,
    152: torchvision.models.resnet152,
}


class Resnet4Channel(nn.Module):
    def __init__(self, encoder_depth=34, pretrained=True, num_classes=28, copy_extra_channel=False, adjust_first_layer=False):
        super().__init__()

        encoder = RESNET_ENCODERS[encoder_depth](pretrained=pretrained)
        
        # we initialize this conv to take in 4 channels instead of 3
        # we keeping corresponding weights and initializing new weights with zeros
        # this trick taken from https://www.kaggle.com/iafoss/pretrained-resnet34-with-rgby-0-460-public-lb
        w = encoder.conv1.weight
        self.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        if copy_extra_channel:
            to_concat = w[:, :1, :, :].clone() 
        else:
            to_concat = torch.zeros(64,1,7,7)
        self.conv1.weight = nn.Parameter(torch.cat((w,to_concat),dim=1) * (0.75 if adjust_first_layer else 1.))
        
        self.bn1 = encoder.bn1
        self.relu = nn.ReLU(inplace=True) 
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = encoder.layer1
        self.layer2 = encoder.layer2
        self.layer3 = encoder.layer3
        self.layer4 = encoder.layer4
        
        self.avgpool = encoder.avgpool
        self.fc = nn.Linear(512 * (1 if encoder_depth==34 else 4), num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
bs = 64

In [ ]:
path = Path('/kaggle/input/')

In [ ]:
df = pd.read_csv(path/'train.csv')
df.head()

In [ ]:
np.random.seed(42)
src = (ImageItemList.from_csv(path, 'train.csv', folder='train', suffix='.png')
       .random_split_by_pct(0.2)
       .label_from_df(sep=' ',  classes=[str(i) for i in range(28)]))

In [ ]:
src.train.x.create_func = open_4_channel
src.train.x.open = open_4_channel

In [ ]:
src.valid.x.create_func = open_4_channel
src.valid.x.open = open_4_channel

In [ ]:
test_ids = list(sorted({fname.split('_')[0] for fname in os.listdir(path/'test')}))

In [ ]:
test_fnames = [path/'test'/test_id for test_id in test_ids]

In [ ]:
test_fnames[:5]

In [ ]:
src.add_test(test_fnames, label='0');

In [ ]:
src.test.x.create_func = open_4_channel
src.test.x.open = open_4_channel

In [ ]:
protein_stats = ([0.08069, 0.05258, 0.05487, 0.08282], [0.13704, 0.10145, 0.15313, 0.13814])

In [ ]:
trn_tfms,_ = get_transforms(do_flip=True, flip_vert=True, max_rotate=30., max_zoom=1,
                      max_lighting=0.05, max_warp=0.)

In [ ]:
data = (src.transform((trn_tfms, _), size=224)
        .databunch(num_workers=0).normalize(protein_stats))

In [ ]:
data.show_batch(rows=3, figsize=(12,9))

In [ ]:
def resnet50(pretrained):
    return Resnet4Channel(encoder_depth=50)

In [ ]:
# copied from https://github.com/fastai/fastai/blob/master/fastai/vision/learner.py
def _resnet_split(m): return (m[0][6],m[1])

In [ ]:
f1_score = partial(fbeta, thresh=0.2, beta=1)

In [ ]:
learn = create_cnn(
    data,
    resnet50,
    cut=-2,
    split_on=_resnet_split,
    loss_func=F.binary_cross_entropy_with_logits,
    path=path,    
    metrics=[f1_score],
    model_dir="/tmp/models/"
)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 3e-2

In [ ]:
learn.fit_one_cycle(1, slice(lr))

In [ ]:
learn.save('stage-1-rn50-test')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, slice(3e-5, lr/5))

In [ ]:
learn.save('stage-2-rn50-test')

In [ ]:
preds,_ = learn.get_preds(DatasetType.Test)

In [ ]:
pred_labels = [' '.join(list([str(i) for i in np.nonzero(row>0.2)[0]])) for row in np.array(preds)]
df = pd.DataFrame({'Id':test_ids,'Predicted':pred_labels})
df.to_csv('submission.csv', header=True, index=False)